# Uji coba sistem pakar

In [4]:
import tkinter as tk
from tkinter import messagebox

# Definisikan data gejala, penyakit, dan hama dengan faktor kepastian
gejala_list = [
    "daun_kuning",
    "daun_berbintik",
    "akar_busuk",
    "bunga_mengerut",
    "tanaman_layu",
    "daun_berlubang"
]

penyakit_dict = {
    "penyakit_busuk_akar": {
        "gejala": ["akar_busuk", "tanaman_layu"],
        "certainty": [0.9, 0.8]
    },
    "penyakit_bintik_daun": {
        "gejala": ["daun_berbintik", "bunga_mengerut"],
        "certainty": [0.85, 0.75]
    },
    "penyakit_layu_bakteri": {
        "gejala": ["daun_kuning", "tanaman_layu"],
        "certainty": [0.8, 0.7]
    }
}

hama_dict = {
    "hama_ulat": {
        "gejala": ["daun_berlubang", "daun_berbintik"],
        "certainty": [0.9, 0.7]
    },
    "hama_kutu_daun": {
        "gejala": ["daun_kuning", "bunga_mengerut"],
        "certainty": [0.85, 0.8]
    }
}

case_base = [
    {
        "gejala": ["daun_kuning", "tanaman_layu"],
        "diagnosis": "penyakit_layu_bakteri",
        "certainty": 0.8
    },
    {
        "gejala": ["akar_busuk", "tanaman_layu"],
        "diagnosis": "penyakit_busuk_akar",
        "certainty": 0.9
    },
    {
        "gejala": ["daun_berlubang", "daun_berbintik"],
        "diagnosis": "hama_ulat",
        "certainty": 0.85
    },
    {
        "gejala": ["daun_kuning", "bunga_mengerut"],
        "diagnosis": "hama_kutu_daun",
        "certainty": 0.8
    }
]

def hitung_certainty(gejala_input, gejala_target, certainty_target):
    total_certainty = 0
    match_count = 0
    
    for gejala, certainty in zip(gejala_target, certainty_target):
        if gejala in gejala_input:
            total_certainty += certainty
            match_count += 1
    
    if match_count == 0:
        return 0
    return total_certainty / match_count

def hitung_kesamaan(gejala_input, gejala_kasus):
    kesamaan = len(set(gejala_input) & set(gejala_kasus)) / len(set(gejala_kasus))
    return kesamaan

def cbr_diagnosis(gejala_input):
    best_case = None
    highest_similarity = 0
    
    for case in case_base:
        similarity = hitung_kesamaan(gejala_input, case["gejala"])
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_case = case
    
    if best_case:
        return best_case["diagnosis"], best_case["certainty"]
    return None, 0

def deteksi_penyakit_hama(gejala_input):
    hasil = {
        "penyakit": None,
        "hama": None,
        "certainty_penyakit": 0,
        "certainty_hama": 0
    }
    
    # Menggunakan rule-based dan certainty
    for penyakit, data_penyakit in penyakit_dict.items():
        certainty = hitung_certainty(gejala_input, data_penyakit["gejala"], data_penyakit["certainty"])
        if certainty > hasil["certainty_penyakit"]:
            hasil["penyakit"] = penyakit
            hasil["certainty_penyakit"] = certainty
    
    for hama, data_hama in hama_dict.items():
        certainty = hitung_certainty(gejala_input, data_hama["gejala"], data_hama["certainty"])
        if certainty > hasil["certainty_hama"]:
            hasil["hama"] = hama
            hasil["certainty_hama"] = certainty
    
    # Menggunakan case-based reasoning
    cbr_diagnosa, cbr_certainty = cbr_diagnosis(gejala_input)
    
    if cbr_diagnosa:
        if "penyakit" in cbr_diagnosa:
            if cbr_certainty > hasil["certainty_penyakit"]:
                hasil["penyakit"] = cbr_diagnosa
                hasil["certainty_penyakit"] = cbr_certainty
        elif "hama" in cbr_diagnosa:
            if cbr_certainty > hasil["certainty_hama"]:
                hasil["hama"] = cbr_diagnosa
                hasil["certainty_hama"] = cbr_certainty
    
    return hasil

# TAMPILAN UI

In [5]:
# Membuat antarmuka pengguna dengan tkinter
class PakarUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistem Pakar Deteksi Penyakit pada Bawang Merah")
        self.root.geometry("450x600")
        self.root.configure(bg="black")

        # Membuat judul
        self.title_label = tk.Label(root, text="Sistem Pakar Deteksi Penyakit\npada Bawang Merah", font=("Helvetica", 16, "bold"), pady=10, fg="white", bg="black")
        self.title_label.pack()

        # Frame untuk gejala
        self.gejala_frame = tk.Frame(root, bg="black")
        self.gejala_frame.pack(pady=10)

        self.gejala_vars = {}
        tk.Label(self.gejala_frame, text="Pilih Gejala yang Ditemukan:", font=("Helvetica", 12, "bold"), fg="white", bg="black").pack(anchor=tk.W)

        # Membuat checkbox untuk setiap gejala
        for gejala in gejala_list:
            var = tk.BooleanVar()
            chk = tk.Checkbutton(self.gejala_frame, text=gejala.replace('_', ' ').capitalize(), variable=var, font=("Helvetica", 10), fg="white", bg="black", selectcolor="black", activebackground="black", activeforeground="white")
            chk.pack(anchor=tk.W)
            self.gejala_vars[gejala] = var
        
        # Tombol untuk mendeteksi penyakit atau hama
        self.btn_detect = tk.Button(root, text="Deteksi", command=self.deteksi, font=("Helvetica", 12, "bold"), bg="blue", fg="white", activebackground="blue", activeforeground="white")
        self.btn_detect.pack(pady=20)

        # Frame untuk hasil
        self.hasil_frame = tk.Frame(root, bg="black")
        self.hasil_frame.pack(pady=10)

    def deteksi(self):
        gejala_input = [gejala for gejala, var in self.gejala_vars.items() if var.get()]

        hasil_deteksi = deteksi_penyakit_hama(gejala_input)

        for widget in self.hasil_frame.winfo_children():
            widget.destroy()

        if hasil_deteksi["penyakit"]:
            hasil_text = f"Penyakit: {hasil_deteksi['penyakit']}\nTingkat Kepastian: {hasil_deteksi['certainty_penyakit']:.2f}"
        elif hasil_deteksi["hama"]:
            hasil_text = f"Hama: {hasil_deteksi['hama']}\nTingkat Kepastian: {hasil_deteksi['certainty_hama']:.2f}"
        else:
            hasil_text = "Gejala yang diamati tidak cocok dengan penyakit atau hama yang diketahui."

        hasil_label = tk.Label(self.hasil_frame, text=hasil_text, font=("Helvetica", 12, "bold"), fg="white", bg="black")
        hasil_label.pack()

# Menjalankan aplikasi tkinter
root = tk.Tk()
app = PakarUI(root)
root.mainloop()